In [1]:
from robobrowser import RoboBrowser
import plotly.plotly as py
import pandas as pd
import networkx as nx
from cachier import cachier
import geocoder

browser = RoboBrowser(history=True, parser='html5lib')
base_url = 'https://flightaware.com/live/fleet/'
query = 'JAL' ## 
airlines = "LAN", "JAL", "DLH", "AAL", "CCA", 'AFR', 'IGO', 'MSR'
df = pd.DataFrame()

for query in airlines:
    print(query)
    for i in range(2):
        url = base_url + query + "?;offset=" + str(i*20)
        browser.open(url)
        table=browser.select('.prettyTable')[0]
        t=table.__str__()
        if 'Sorry' in t:
                break
        tdf=pd.read_html(t, header=1)[0]
        df = df.append(tdf)
        print(tdf.head(3))

LAN
    Ident  Type                                             Origin  \
0  LAN127  A320                    Cerro Moreno Int'l (ANF / SCFA)   
1  LAN149  A321                                El Loa (CJC / SCCF)   
2  LAN164  A321  Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)   

                                         Destination        Departure  \
0  Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)  Mon 10:56AM -03   
1  Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)  Mon 11:29AM -03   
2                   Diego Aracena Int'l (IQQ / SCDA)  Mon 11:29AM -03   

  EstimatedArrival Time  
0       Mon 12:35PM -03  
1       Mon 01:13PM -03  
2       Mon 01:35PM -03  
JAL
    Ident  Type                             Origin  \
0   JAL18  B788          Narita Int'l (NRT / RJAA)   
1    JAL2  B77W  Tokyo Int'l (Haneda) (HND / RJTT)   
2  JAL228  B738          Kansai Int'l (KIX / RJBB)   

                         Destination        Departure EstimatedArrival Time  
0             Vanco

In [39]:
G = nx.from_pandas_dataframe(df, 'Origin', 'Destination')
centrality=nx.betweenness_centrality(G)
centrality

{"Abu Dhabi Int'l (AUH / OMAA)": 0.0,
 'Albuquerque Intl Sunport (KABQ)': 0.0,
 'Amausi (LKO / VILK)': 0.0,
 'Amerigo Vespucci (Florence) (FLR / LIRQ)': 0.0,
 'Amsterdam Schiphol (AMS / EHAM)': 0.0,
 "Athens Int'l, Eleftherios Venizelos (ATH / LGAV)": 0.0,
 'Auckland (AKL / NZAA)': 0.0,
 'Austin-Bergstrom Intl (KAUS)': 0.0,
 "Bahrain Int'l (BAH / OBBI)": 0.0,
 'Baotou (BAV / ZBOW)': 0.0,
 "Barajas Int'l (MAD / LEMD)": 0.002640561097733443,
 "Barcelona Int'l (BCN / LEBL)": 0.020198371746359373,
 "Beijing Capital Int'l (PEK / ZBAA)": 0.26796926957917633,
 "Beirut Air Base/Rafic Hariri Int'l (Beirut Int'l) (BEY / OLBA)": 0.0,
 "Ben Gurion Int'l (TLV / LLBG)": 0.0,
 "Bengaluru Int'l (BLR / VOBL)": 0.024615465183060653,
 'Biju Patnaik (BBI / VEBS)': 0.0,
 'Bologna (Guglielmo Marconi) (BLQ / LIPE)': 0.0,
 'Boston Logan Intl (KBOS)': 0.01396956443396072,
 "Bucharest Henri Coandă Int'l (OTP / LROP)": 0.0,
 'Budapest Ferenc Liszt International Airport (BUD / LHBP)': 0.0,
 "Cairo Int'l (CAI / HE

In [40]:
from joblib import Memory
memory = Memory(cachedir='tmp', verbose=0)

@memory.cache
def get_latlng(query):
    g=geocoder.google(query)
    if g:
        return g.latlng
    else:
        return None, None
    
great_lines = []
airports = []
airport_data = []
seen=[]
great_lines_2=[]
airports = pd.DataFrame(columns=['label','lat','lon','size'])

    
i=0
for index,row in df.iterrows():
    origin = row.Origin
    destination = row.Destination
    olat, olon = get_latlng(origin)
    dlat, dlon = get_latlng(destination)
    if olat and dlat:
        if origin not in seen:
            seen.append(origin)
            print(origin, olat, olon)
        if destination not in seen:
            seen.append(destination)
            print(destination, dlat, dlon)
        great_lines.append(dict(
            type = 'scattergeo',
            lat = [ olat, dlat ],
            lon = [ olon, dlon ],
            hoverinfo = 'Ident',
                text = 'Ident',
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'rgba(255,0,0,0.2)',
                ),
                )
        )
        airports.loc[i]=[origin, olat, olon, 0.0]
        airports.loc[i+1]= [destination, dlat, dlon, 0.0]
        i+=2
        
for key, item in centrality.items():
    airports.loc[airports.label==key, 'size'] = (item*40)+10
    
airport_markers = [ dict(
        type = 'scattergeo',
        lon = airports['lon'],
        lat = airports['lat'],
        hoverinfo = 'label',
        text = airports['label'],
        mode = 'markers',
        marker = dict( 
            size=airports['size'], 
            color='red',   
        ))]
    
layout = dict(
        title = 'Great lines over globe<br>(Click and drag to rotate)',
        width = 800,
        height = 800,
        showlegend = False, 
        showocean = True,
        oceancolor = '#acf',
        showland = True,
        geo = dict(
            projection = dict( 
                type = 'orthographic',          
            ),
            
        )
    )
    
fig = dict( data=great_lines+airport_markers, layout=layout )
py.iplot( fig, validate=False, filename='d3-globe' )

Cerro Moreno Int'l (ANF / SCFA) -23.4486944 -70.4409348
Comodoro Arturo Merino Benitez Int'l (SCL / SCEL) -33.3911894 -70.79723539999999
El Loa (CJC / SCCF) -22.4989612 -68.90696609999999
Diego Aracena Int'l (IQQ / SCDA) -20.5364084 -70.1805181
Jorge Newbery (AEP / SABE) -34.5580305 -58.4170088
Ministro Pistarini Int'l (EZE / SAEZ) -34.8150044 -58.53482839999999
Jorge Chavez Int'l (LIM / SPJC) -12.0240527 -77.112036
Barajas Int'l (MAD / LEMD) 40.4839361 -3.5679515
Frankfurt Int'l (FRA / EDDF) 50.0379326 8.562151799999999
Auckland (AKL / NZAA) -37.0082476 174.7850358
Mataveri Int'l (IPC / SCIP) -27.158723 -109.4300364
Narita Int'l (NRT / RJAA) 35.7719867 140.3928501
Vancouver Int'l (CYVR) 49.1966913 -123.1815123
Kansai Int'l (KIX / RJBB) 34.4320024 135.2303939
Tokyo Int'l (Haneda) (HND / RJTT) 35.5493932 139.7798386
Singapore Changi (SIN / WSSS) 1.3644202 103.9915308
John F Kennedy Intl (KJFK) 40.6413111 -73.77813909999999
Helsinki-Vantaa (HEL / EFHK) 60.32104159999999 24.9528604
Charle

In [ ]:
from nxpd import draw
from nxpd import nxpdParams
nxpdParams['show'] = 'ipynb'
G.graph['dpi'] = 120
draw(G)

In [41]:
layout = dict(
        title = 'Great lines over globe<br>(Click and drag to rotate)',
        showlegend = False,  
        width = 800,
        height = 800,
        geo = dict(
            
            showland = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            landcolor = '#fff',
            oceancolor = '#eee',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            
        )
    )
    
fig = dict( data=great_lines+airport_markers, layout=layout )
py.iplot( fig, validate=False, filename='d3-globe' )